In [1]:
import sys
import scipy
import numpy as np
import dask.array as da
import dask.dataframe as dd
import pandas as pd
sys.path.insert(0, "../")
from GWASSimulator import GWASSimulator
%load_ext autoreload
%autoreload 2

In [2]:
gs = GWASSimulator("../data/genotype_data/1000G.EUR.QC.22",
                   keep_snps="../data/w_snplist_no_MHC.snplist.bz2")
gs.simulate()

> Reading genotype files...


Mapping files: 100%|██████████| 3/3 [00:00<00:00,  6.52it/s]


> Computing LD matrices...


/Users/szabad/opt/miniconda3/lib/python3.7/site-packages/dask/array/routines.py:272: PerformanceWarning: Increasing number of chunks by factor of 138
  axes=(left_axes, right_axes),


In [3]:
gs.update_ld_regularization(10.)
gs.simulate(n=10000)

In [15]:
gs.get_p_values()[0].compute()

array([0.81618536, 0.04718362, 0.56398163, ..., 0.16610908, 1.00264266,
       0.020099  ])